In [1]:
import numpy as np
import matplotlib.pylab as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import MobileNet

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import pathlib
import zipfile
from shutil import copyfile
import random
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
local_zip = '/content/drive/MyDrive/dataset_foto/indonesian_food.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
!mkdir makanan_indo

In [ ]:
import os
import shutil

root_dir = './makanan_indo'

def create_train_val_dirs(root_path):
    train_dir = os.path.join(root_path, 'training')
    valid_dir = os.path.join(root_path, 'validation')

    classes = ['ayam bakar', 'ayam geprek', 'bakso', 'gado gado', 'mie ayam', 'rendang', 'sate', 'sayur asem', 'soto  ayam']
    subfolders = ['train', 'valid']

    for c in classes:
        for sf in subfolders:
            class_dir = os.path.join(root_path, sf, c)
            os.makedirs(class_dir)

In [ ]:
create_train_val_dirs(root_path=root_dir)

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  files = []
  for filename in os.listdir(SOURCE_DIR):
    file_path = os.path.join(SOURCE_DIR, filename)
    if os.path.getsize(file_path) > 0:
      files.append(filename)
    else:
      print(f"{filename}, filename is zero length, so ignoring.")

  train_data_length = int(len(files) * SPLIT_SIZE)
  valid_data_length = int(len(files) - train_data_length)

  shuffled_data = random.sample(files, len(files))
  train_data = shuffled_data[0:train_data_length]
  valid_data = shuffled_data[-valid_data_length:]

  for filename in train_data:
    source_dir = os.path.join(SOURCE_DIR, filename)
    dest_dir = os.path.join(TRAINING_DIR, filename)
    copyfile(source_dir, dest_dir)

  for filename in valid_data:
    source_dir = os.path.join(SOURCE_DIR, filename)
    dest_dir = os.path.join(VALIDATION_DIR, filename)
    copyfile(source_dir, dest_dir)

In [ ]:
import os
import random
from shutil import copyfile

AYAMBAKAR_SOURCE_DIR = '/content/indonesian_food/ayam bakar'
AYAMGEPREK_SOURCE_DIR = '/content/indonesian_food/ayam geprek'
BAKSO_SOURCE_DIR = '/content/indonesian_food/bakso'
MIEAYAM_SOURCE_DIR ='/content/indonesian_food/mie ayam'
GADOGADO_SOURCE_DIR = '/content/indonesian_food/gado gado'
RENDANG_SOURCE_DIR = '/content/indonesian_food/rendang'
SATE_SOURCE_DIR = '/content/indonesian_food/sate'
SAYURASEM_SOURCE_DIR = '/content/indonesian_food/sayur asem'
SOTOAYAM_SOURCE_DIR = '/content/indonesian_food/soto ayam'


TRAINING_DIR = "/content/makanan_indo/train"
VALIDATION_DIR = "/content/makanan_indo/valid"

categories = ['ayam bakar', 'ayam geprek', 'bakso', 'gado gado', 'mie ayam', 'rendang', 'sate', 'sayur asem', 'soto  ayam']

for category in categories:
    training_category_dir = os.path.join(TRAINING_DIR, category)
    validation_category_dir = os.path.join(VALIDATION_DIR, category)
    if len(os.listdir(training_category_dir)) > 0:
        for file in os.scandir(training_category_dir):
            os.remove(file.path)
    if len(os.listdir(validation_category_dir)) > 0:
        for file in os.scandir(validation_category_dir):
            os.remove(file.path)

# Define proportion of images used for training
split_size = 0.8

# Run the function for each category
for category in categories:
    if category == 'ayam bakar':
      category_source_dir = AYAMBAKAR_SOURCE_DIR
    elif category == 'ayam geprek':
      category_source_dir = AYAMGEPREK_SOURCE_DIR
    elif category == 'bakso':
      category_source_dir = BAKSO_SOURCE_DIR
    elif category == 'gado gado':
      category_source_dir = GADOGADO_SOURCE_DIR
    elif category == 'mie ayam':
      category_source_dir = MIEAYAM_SOURCE_DIR
    elif category == 'rendang':
      category_source_dir = RENDANG_SOURCE_DIR 
    elif category == 'sate':
      category_source_dir = SATE_SOURCE_DIR
    elif category == 'sayur asem':
      category_source_dir = SAYURASEM_SOURCE_DIR
    elif category == 'soto ayam':
      category_source_dir = SOTOAYAM_SOURCE_DIR
    training_category_dir = os.path.join(TRAINING_DIR, category)
    validation_category_dir = os.path.join(VALIDATION_DIR, category)
    split_data(category_source_dir, training_category_dir, validation_category_dir, split_size)


In [ ]:
class ImageGenerator:
    def __init__(self, train_dir, validation_dir):
        self.train_dir = train_dir
        self.validation_dir = validation_dir

    def create_generators(self):
        train_datagen = ImageDataGenerator(
            rescale=1./255.,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )

        test_datagen = ImageDataGenerator(rescale=1.0/255.)

        train_generator = train_datagen.flow_from_directory(
            self.train_dir,
            batch_size=20,
            class_mode='categorical',
            target_size=(224, 224)
        )

        validation_generator = test_datagen.flow_from_directory(
            self.validation_dir,
            batch_size=20,
            class_mode='categorical',
            target_size=(224, 224)
        )

        return train_generator, validation_generator

In [ ]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
train_dir = TRAINING_DIR
valid_dir = VALIDATION_DIR 
callbacks = myCallback()
image_gen = ImageGenerator(train_dir, valid_dir)
train_generator, validation_generator = image_gen.create_generators()

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation = 'relu'),
    Dropout(0.3),
    Dense(9, activation='softmax'),
])

model.compile(optimizer = Adam(learning_rate = 0.003),loss = 'categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 20,
                    callbacks=callbacks)

Found 1433 images belonging to 9 classes.
Found 359 images belonging to 9 classes.
Epoch 1/20
72/72 [==============================] - 54s 716ms/step - loss: 1.7716 - accuracy: 0.3678 - val_loss: 0.8100 - val_accuracy: 0.7409
Epoch 2/20
72/72 [==============================] - 51s 705ms/step - loss: 1.1666 - accuracy: 0.5387 - val_loss: 0.7034 - val_accuracy: 0.7437
Epoch 3/20
72/72 [==============================] - 50s 697ms/step - loss: 1.0072 - accuracy: 0.6127 - val_loss: 0.5705 - val_accuracy: 0.7716
Epoch 4/20
72/72 [==============================] - 49s 687ms/step - loss: 0.9287 - accuracy: 0.6274 - val_loss: 0.5865 - val_accuracy: 0.7521
Epoch 5/20
72/72 [==============================] - 51s 704ms/step - loss: 0.8752 - accuracy: 0.6525 - val_loss: 0.6010 - val_accuracy: 0.7465
Epoch 6/20
72/72 [==============================] - 49s 686ms/step - loss: 0.8348 - accuracy: 0.6595 - val_loss: 0.5827 - val_accuracy: 0.7772
Epoch 7/20
72/72 [==============================] - 51s 713

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

class_names = ['ayam bakar', 'ayam geprek', 'bakso', 'gado gado', 'mie ayam', 'rendang', 'sate', 'sayur asem', 'soto  ayam']  

for fn in uploaded.keys():
    # Predicting images
    path = '/content/' + fn
    img = load_img(path, target_size=(224, 224))
  
    x = img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
  
    classes = model.predict(images, batch_size=10)
  
    predicted_class_index = np.argmax(classes[0])  
  
    predicted_class = class_names[predicted_class_index]  
  
    print(fn + " is a " + predicted_class)


Saving rendang (59).jpg to rendang (59).jpg
1/1 [==============================] - 0s 28ms/step
rendang (59).jpg is a rendang


In [ ]:
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)

In [ ]:
mode = "Speed" 
export_dir = 'saved_model/1'

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)